In [1]:
!pip install -r ./requirements.txt -q

In [2]:
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.function_step import step
from sagemaker import get_execution_role, Session
import logging

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
from steps.fetch import fetch
from steps.preprocess import preprocess
from steps.train import train
from steps.eval import eval
from steps.deploy import deploy

In [4]:
sagemaker_session = Session()
role = get_execution_role()

default_bucket = sagemaker_session.default_bucket()
session = PipelineSession(boto_session=sagemaker_session.boto_session, default_bucket=default_bucket)
pipeline_name = 'stock-pipeline'
default_ticker = "SPY"
years_of_data_to_fetch = 1

In [5]:
# @step(
#     instance_type="ml.m5.large"
# )
# def train(data):
#     # retrieve preprocessed csv data
#     # train model
#     # save model to s3 bucket /model with postfix of date and RMSE in filename
#     logging.info(f'data should be output of preprocess function: {data}')
#     return "stub_train"

In [6]:
# @step(
#     instance_type="ml.m5.large"
# )
# def eval(data):
#     # load current model file and latest
#     # extract model performance from RSME in filename
#     # if model is better than current
#     # pass True to deploy phase (Use conditional to assess when to run deploy)
#     logging.info(f'data should be output of preprocess function: {data}')
#     return False

In [7]:
# @step(
#     instance_type="ml.m5.large"
# )
# def deploy(data):
#     # if latest_model is different than current
#     # update deployment to use latest model
#     logging.info(f'data should be output of train function: {data}')
#     return "stub_eval"

In [6]:
fetch_result = fetch(default_ticker, years_of_data_to_fetch)
print(fetch_result)

INFO:steps.utils:Successfully fetched data for SPY
INFO:steps.utils:Raw data saved to s3://sagemaker-us-east-1-971173012767/raw/SPY_raw_data.csv


s3://sagemaker-us-east-1-971173012767/raw/SPY_raw_data.csv


In [7]:
preprocess_result = preprocess(fetch_result)
print(preprocess_result)

INFO:steps.utils:Starting preprocessing with data from s3://sagemaker-us-east-1-971173012767/raw/SPY_raw_data.csv
INFO:steps.utils:Successfully read raw data from s3://sagemaker-us-east-1-971173012767/raw/SPY_raw_data.csv
INFO:steps.utils:Data preprocessing completed
INFO:steps.utils:Preprocessed data saved to s3://sagemaker-us-east-1-971173012767/processed/processed/SPY_processed_data.csv


s3://sagemaker-us-east-1-971173012767/processed/processed/SPY_processed_data.csv


In [8]:
train_result = train(preprocess_result)
print(train_result)

INFO:steps.utils:Starting training with data from s3://sagemaker-us-east-1-971173012767/processed/processed/SPY_processed_data.csv
INFO:steps.utils:Loaded preprocessed data with shape (250, 4)
INFO:steps.utils:Epoch 1/10, Loss: 0.40415561695893604
INFO:steps.utils:Epoch 2/10, Loss: 0.25259753316640854
INFO:steps.utils:Epoch 3/10, Loss: 0.09453133183221023
INFO:steps.utils:Epoch 4/10, Loss: 0.03746751012901465
INFO:steps.utils:Epoch 5/10, Loss: 0.02119459956884384
INFO:steps.utils:Epoch 6/10, Loss: 0.020061183410386246
INFO:steps.utils:Epoch 7/10, Loss: 0.016886258653054636
INFO:steps.utils:Epoch 8/10, Loss: 0.011630564772834381
INFO:steps.utils:Epoch 9/10, Loss: 0.01056354803343614
INFO:steps.utils:Epoch 10/10, Loss: 0.008187109914918741
INFO:steps.utils:Training completed with RMSE: 12.677477423538493
INFO:steps.utils:Model saved to s3://sagemaker-us-east-1-971173012767/model/model_20250221_23_rmse_12.6775.safetensors


('s3://sagemaker-us-east-1-971173012767/model/model_20250221_23_rmse_12.6775.safetensors', 12.677477423538493)


In [9]:
eval_result = eval(train_result)
print(eval_result)

INFO:steps.utils:Evaluating model from s3://sagemaker-us-east-1-971173012767/model/model_20250221_23_rmse_12.6775.safetensors with RMSE: 12.677477423538493
INFO:steps.utils:Loaded model from s3://sagemaker-us-east-1-971173012767/model/model_20250221_23_rmse_12.6775.safetensors
INFO:steps.utils:No current model found or error loading s3://sagemaker-us-east-1-971173012767/model/current_model.safetensors: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.. Assuming new model is better.
INFO:steps.utils:New RMSE: 12.677477423538493, Current RMSE: inf, Deploy: True


(True, 's3://sagemaker-us-east-1-971173012767/model/model_20250221_23_rmse_12.6775.safetensors')


In [10]:
deploy_result = deploy(eval_result)
print(deploy_result)

INFO:steps.utils:Received eval output: deploy_flag=True, model_s3_path=s3://sagemaker-us-east-1-971173012767/model/model_20250221_23_rmse_12.6775.safetensors
INFO:steps.utils:Loaded model from s3://sagemaker-us-east-1-971173012767/model/model_20250221_23_rmse_12.6775.safetensors
INFO:steps.utils:Model artifacts uploaded to s3://sagemaker-us-east-1-971173012767/model/model.tar.gz
INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-971173012767/model/model.tar.gz), script artifact (/home/sagemaker-user/aai-540_stock_trading_ai/cloud/steps), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-971173012767/pytorch-inference-2025-02-21-23-40-58-073/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2025-02-21-23-40-58-670
INFO:sagemaker:Creating endpoint-config with name stock-pipeline-endpoint-20250221-234058
INFO:sagemaker:Creating endpoint with name stock-pipeline-endpoint

------!

INFO:steps.utils:Model deployed to endpoint: stock-pipeline-endpoint-20250221-234058
INFO:steps.utils:Updated current model at s3://sagemaker-us-east-1-971173012767/model/current_model.safetensors


stock-pipeline-endpoint-20250221-234058


In [7]:
pipeline = Pipeline(
    name=pipeline_name,
    steps=[fetch_result, preprocess_result, train_result, eval_result, deploy_result],
    sagemaker_session=session,
)

In [8]:
pipeline.upsert(role)

2025-02-21 20:50:38,084 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-80b1c5ef-0734-4a54-9399-a435f0087fff/2025-02-21-20-50-37-995/function
2025-02-21 20:50:38,258 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-80b1c5ef-0734-4a54-9399-a435f0087fff/2025-02-21-20-50-37-995/arguments


ValueError: Invalid dependencies provided: "cloud/"

In [ ]:
pipeline.start()

## Verify after successful pipeline run

In [16]:
from steps.deploy import deploy
import boto3
import json
import torch
import numpy as np
from datetime import datetime
import logging

In [ ]:
# Set to endpoint name
endpoint_name="stock-pipeline-endpoint-20250221-234058"

In [ ]:
sequence_length = 30
input_data = np.random.rand(sequence_length, 4).tolist()  
# Ensure the input matches the expected shape: [seq_len, 4]
# endpoint_name="https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/stock-pipeline-endpoint-20250221-234058/invocatio
endpoint_name="stock-pipeline-endpoint-20250221-234058"

# Convert to JSON
input_json = json.dumps(input_data)

# Call the endpoint
try:
    runtime = boto3.client('sagemaker-runtime')
    print(f"Invoking endpoint: {endpoint_name}")
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=input_json
    )
    
    # Parse the response
    result = json.loads(response['Body'].read().decode())
    # logger.info(f"Prediction: {result}")
    print(f"Prediction from endpoint {endpoint_name}: {result}")

except Exception as e:
    print(f"Error invoking endpoint: {e}")
    raise e